In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
pip install glob2
import glob
glob.glob('c:\\data\\foods\\**\\*.jpg', recursive=True)

In [ ]:
import glob
from PIL import Image
import glob

all_images = []
all_labels = []
img_size = (96, 96)
for f in glob.glob("c:\\data\\foods\\**\\*.jpg", recursive=True):
    arr = split("\\")
    print(arr)
    img = Image.open(f)
    img_resize.append(img_resize)
    label = 0
    print(arr)
    if arr[4] == "Chicken":
        label = 0
    elif arr[4] == "Dolsot":
        label = 1
    elif arr[4] == "Jeyug":
        label = 2
    elif arr[4] == "Kimchi":
        label = 3
    elif arr[4] == "Samgyeb":
        label = 4
    elif arr[4] == "Soybean":
        label = 5
    print(label)
    all_labels.append(label)

In [ ]:
import numpy as np

X = np.empty((1, img_size[0], img_size[1], 3))
for img in all_images:
    X = np.vstack((X, np.array(img).reshape(1, img_size[0], img_size[1], 3)))
X.shape

In [ ]:
X[0].shape

In [ ]:
X = np.delete(X, (0), axis=0)

In [ ]:
X.shape

In [ ]:
y = np.array(all_labels)
y

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(all_images[0])
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True,  random_state=0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import pandas as pd

pd.DataFrame(y_train).value_counts()

In [ ]:
X_train = X_train.reshape(-1, img_size[0] * img_size[1] * 3)
X_test = X_test.reshape(-1, img_size[0] * img_size[1] * 3)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train[0]

In [ ]:
from tensorflow.keras import utils

y_train_backup = y_train

y_test_backup = y_test

y_train = utils.to_categorical(y_train, 6)

y_test = utils.to_categorical(y_test, 6)

In [ ]:
y_train[:5]

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

# VGG19 모델

# 1. 전이학습

base_model = VGG19(
    weights="imagenet",
    input_shape=(96, 96, 3),
    include_top=False,
)

model = VGG19(input_shape=(96, 96, 3), include_top=False, weights="imagenet")

model.trainable = False

t_model = tf.keras.models.Sequential(
    [
        model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(6, activation="softmax"),
    ]
)

t_model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    loss="categorical_crossentropy",
    metrics="accuracy",
)

t_model.summary()

In [ ]:
# 2. Fine-Tuning

t_model.trainable = True

t_model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
    loss="categorical_crossentropy",
    metrics="accuracy",
)

t_model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    "d:/data/models/food_best2.h5",
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode="max",
)

hist = t_model.fit(
    X_train,
    y_train,
    batch_size=32,
    validation_split=0.2,
    epochs=30,
    callbacks=[checkpoint],
)

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(hist.history["loss"])

plt.plot(hist.history["val_loss"])

plt.legend(["loss", "val_loss"])

plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(hist.history["accuracy"])

plt.plot(hist.history["val_accuracy"])

plt.legend(["accuracy", "val_accuracy"])

plt.grid()

plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("d:/data/models/food_best2.h5")

scores = model.evaluate(X_train, y_train, verbose=1)

print(scores)

print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))  # 학습용

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=1)

print(scores)

print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))  # 검증용

# cnn 60%

# transfer learning 69%

# fine tuning 85%

In [ ]:
pred = model.predict(X_test)

print(pred)

print(np.argmax(pred, axis=1))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_backup, np.argmax(pred, axis=1)))